<a href="https://colab.research.google.com/github/jonesavelino/idea-tool/blob/main/IDEA_RodaModeloNEReRE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETAPA 3: Rodar NER + RE
## Objetivo: Submeter um conjunto de sentenças de textos ao modelo pré-treinado no domínio e extrair as NER e RE
## Entrada: .../avaliar/inputs.txt
# Pré-requisitos:
### Modelo NER treinado: NER_MODEL_PATH = ".../NER/outputs/model-best/"
### Modelo RE treinado: RE_MODEL_PATH = ".../RE/rel_component/training/model-best"
### Recuperar o MAP_LABELS utilizado no RE
## Etapas:
### Etapa 1: Montar o Drive
### Etapa 2: Configurações básicas do Spacy
### Etapa 3: Definir parâmetros de extração
### Etapa 4: Submeter o texto ao modelo de NER e RE já treinado no domínio



In [ ]:
#***************************************************************************************************************************
# Etapa 1: Montar o Drive
#***************************************************************************************************************************
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#***************************************************************************************************************************
# Etapa 2: Configurações básicas do Spacy
# OBS: Estão sendo utilizadas funções externas dos arquivos: rel_pipe.py, rel_model.py e auxiliares.py
#***************************************************************************************************************************
!pip install -U spacy
!python -m spacy download pt_core_news_sm
!pip install spacy-transformers

import spacy
import pandas as pd
# import para rodar pacotes python .py
import sys
# acessar a pasta acima da qual o .py está localizado
# Exemplo: /content/drive/MyDrive/Doutorado/IDEA/RE/rel_component/scripts.rel_pipe.py
# pra usar é só  scripts.rel_pipe

import random
import typer
from pathlib import Path
from spacy.tokens import DocBin, Doc
from spacy.training.example import Example
sys.path.append('/content/drive/MyDrive/Doutorado/IDEA/RE/rel_component')
from scripts.rel_pipe import make_relation_extractor, score_relations
from scripts.rel_model import create_relation_model, create_classification_layer, create_instances, create_tensors
sys.path.append('/content/drive/MyDrive/Doutorado')
from IDEA.auxiliares import *



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 56.8 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.2
    Uninstalling spacy-3.5.2:
      Successfully uninstalled spacy-3.5.2
2023-05-24 10:18:17.322543: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-24 10:18:18.681839: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 81.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load th

In [ ]:
#***************************************************************************************************************************
# Etapa 3: Definir parâmetros de extração
#***************************************************************************************************************************

######################## CONFIGURACOES DA EXTRACAO ########################
###########################################################################

# Passar os modelos de NER e RE treinados

# Local do model-best de NER treinado
NER_MODEL_PATH = "/content/drive/MyDrive/Doutorado/IDEA/NER/outputs/model-best/"
# Local do model-best de RE treinado
RE_MODEL_PATH = "/content/drive/MyDrive/Doutorado/IDEA/RE/rel_component/training/model-best"
##############################################################################################
#ANTISARP_NER_MODEL_PATH = "output-ner-antisarp/model-best"
#ANTISARP_RE_MODEL_PATH = "rel_component_antisarp/training/model-best"
#NER_MODEL_PATH = SCIERC_NER_MODEL_PATH
#RE_MODEL_PATH = SCIERC_NER_MODEL_PATH

LIMIAR = 0.2

QUEBRA_SENTS = False


# Apontar na constante LABELS qual o MAP_LABELS associado
LABELS = 'IDEA'
# LABELS = 'ANTISARP'
# LABELS = 'SCIERC'
# LABELS = 'IDEA'

# Esse MAP_LABELS é definido no notebook IDEA-ConverterDoccanoSpacy

# Labels SICERC
MAP_LABELS_SCIERC = {
    "USED-FOR": "USED-FOR",
    "COREF": "COREF",
    "CONJUNCTION": "CONJUNCTION",
    "HYPONYM-OF": "HYPONYM-OF",
    "COMPARE": "COMPARE",
    "PART-OF": "PART-OF"
}
# Labels ANTISARP
MAP_LABELS_ANTISARP = {
    "usada_para": "usada_para"
}

# Labels IDEA
MAP_LABELS_IDEA = {
        "tratado_em": "tratado_em",
        "composto_de": "composto_de",
        "instancia_de": "instancia_de",
        "finalidade_de": "finalidade_de",
        "aplicado_em": "aplicado_em",
        "ocorre_em": "ocorre_em",
        "correferencia_de": "correferencia_de",
        "responsavel_por": "responsavel_por",
        "complementado_por": "complementado_por",
        "topico_de": "topico_de",
        "capacidade_de": "capacidade_de",
        "tipo_de": "tipo_de",
        "definido_por": "definido_por",
        "referenciado_a": "referenciado_a",
        "capacidade_de": "capacidade_de",
        "ocorre_em": "ocorre_em"
    }



if LABELS == 'SCIERC':
    MAP_LABELS = MAP_LABELS_SCIERC
elif LABELS == 'ANTISARP':
    MAP_LABELS = MAP_LABELS_ANTISARP
elif LABELS == 'IDEA':
    MAP_LABELS = MAP_LABELS_IDEA

print("Configuracoes:")
Sent = 'Sim' if QUEBRA_SENTS else 'Nao'
print(f"   {LABELS} -- Limiar = {LIMIAR} -- Quebra em sentenças? {Sent}")
###########################################################################

Configuracoes:
   IDEA -- Limiar = 0.2 -- Quebra em sentenças? Nao


##Etapa: 4: Submeter o texto ao modelo de NER e RE já treinado no domínio

### Passo 1: Obter os modelos de NER e RE treinados no domínio (nlp e nlp2)
### Passo 2: Obter dados de entrada (textos para avaliar): inputs.txt
### Passo 3: Submeter os textos de inputs.txt ao modelo NER treinado
### Passo 4: Identificar as NER e armazenar em uma estrutura de dados DICT
### Passo 5: Com base nas NER obtidas, identificar as relações no modelo RE treinado
### Passo 6: Gerar os arquivos de saída (entidades.n3 e relacoes.n3): com as entidades e relações

In [ ]:
#***************************************************************************************************************************
# Passo 1: Obter os modelos de NER e RE treinados no domínio (nlp e nlp2)
#***************************************************************************************************************************
# Modelo NER
nlp = spacy.load(NER_MODEL_PATH)
nlp.add_pipe('sentencizer')  # ADICIONEI DEPOIS

# Modelo RE
nlp2 = spacy.load(RE_MODEL_PATH)

#***************************************************************************************************************************
# Passo 2: Obter dados de entrada (textos para avaliar): inputs.txt
#***************************************************************************************************************************
cols1 = ['sentenca']
text = pd.DataFrame(columns=cols1)
text = pd.read_csv('/content/drive/MyDrive/Doutorado/IDEA/avaliar/inputs.txt', header=None, sep=";", names=cols1)

#text = ['''Os elementos do poder de combate terrestre representam a essência das capacidades que a F Ter emprega em situações – sejam de guerra ou de não guerra. São eles: Liderança, Informações e as Funções de Combate – Comando e Controle, Movimento e Manobra, Inteligência, Fogos, Logística e Proteção. ''']
print(text.head())


#***************************************************************************************************************************
# Passo 3: Submeter os textos de inputs.txt ao modelo NER treinado e
# Passo 4: Identificar as NER e armazenar em uma estrutura de dados DICT
#***************************************************************************************************************************
id_ent_cont= 0 # variável que conta as entidades nomeadas encontradas
id_rel_cont = 0 # variável que conta as relações encontradas

dim_ents = [] # vetor que armazena as entidades encontradas
fato_ents = []

fato_rels = []
dim_rels = []

ent_id_start = {}  # Dicionario para identificar (depois, no armazenamento das relacoes) o id da entidade pelo seu start

for cont_docum, doc in enumerate(nlp.pipe(text.sentenca, disable=["tagger"])):
  #print('\nTexto:', doc.text)
  print('\nEntidades:')
  for e in doc.ents:
    nr_ent = recupera_entidade(dim_ents, e.text, e.label_)
    #print(f'   {e.start_char}\t{e.text} [{e.label_}]')
    if nr_ent == -1:
      print(e.text, "não encontrada. Inserindo...")
      print(id_ent_cont, e.text, e.label_)
      dim_ents.append([id_ent_cont, e.text, e.label_])
      #fato_ents.append([id_doc, id_ent_cont, 1])
    else:
      print("já tinha")
      #fato_ents.append([id_doc, nr_ent, 1])
    ent_id_start[e.start_char] = id_ent_cont  # So posso fazer isso porque modelo nao aceita overlap
    id_ent_cont += 1

  print('\nTodas entidades já foram processadas...')
  print(ent_id_start)

#***************************************************************************************************************************
# Passo 5: Com base nas NER obtidas, identificar as relações no modelo RE treinado
#***************************************************************************************************************************
  for name, proc in nlp2.pipeline:
    doc = proc(doc)
    print('\nRelações:')
    if (QUEBRA_SENTS):
      # Aqui, dividimos o parágrafo em frases e aplicamos a extração de relação
      # para cada par de entidades encontradas em cada sentença..
      for value, rel_dict in doc._.rel.items():
        for sent in doc.sents:
          for e in sent.ents:
            for b in sent.ents:
              #print('>> e.start_char:', e.start_char, 'b.start_char:', b.start_char)
              id_ent_e = ent_id_start[e.start_char]  # Recuperando id da entidade head pelo seu start
              id_ent_b = ent_id_start[b.start_char]  # Recuperando id da entidade tail pelo seu start
              if e.start == value[0] and b.start == value[1]:
                for label in MAP_LABELS.values():
                  if rel_dict[label] >= LIMIAR:
                    print(f"     {e.text} --{label}-- {b.text} [{rel_dict[label]:.2%}]")
                    dim_rels.append([id_rel_cont, label])
                    #fato_rels.append([id_doc, id_ent_e, id_ent_b, id_rel_cont])
                    id_rel_cont += 1
    else:
      # Sem quebrar em sentenças
      for value, rel_dict in doc._.rel.items():
        for e in doc.ents:
          for b in doc.ents:
            #print('>> e.start_char:', e.start_char, 'b.start_char:', b.start_char)
            id_ent_e = ent_id_start[e.start_char]  # Recuperando id da entidade head pelo seu start
            id_ent_b = ent_id_start[b.start_char]  # Recuperando id da entidade tail pelo seu start
            if e.start == value[0] and b.start == value[1]:
              for label in MAP_LABELS.values():
                if rel_dict[label] >= LIMIAR:
                  print(f"     {e.text} --{label}-- {b.text} [{rel_dict[label]:.2%}]")
                  dim_rels.append([id_rel_cont, e.text, label,b.text])
                  #fato_rels.append([id_doc, id_ent_e, id_ent_b, id_rel_cont])
                  id_rel_cont += 1
    print('\n')
    #poder de combate --correferencia_de-- PODER DE COMBATE [94.22%]

                                            sentenca
0                                     EB20-MF-10.102
1                         DOUTRINA MILITAR TERRESTRE
2  ELEMENTOS DO PODER DE COMBATE - Os elementos d...

Entidades:
	Verificando: EB20-MF-10.102 entidade
EB20-MF-10.102 não encontrada. Inserindo...
0 EB20-MF-10.102 entidade

Todas entidades já foram processadas...
{0: 0}

Relações:


ℹ Could not determine any instances in doc - returning doc as is.

Relações:



Entidades:
	Verificando: DOUTRINA MILITAR TERRESTRE entidade
DOUTRINA MILITAR TERRESTRE não encontrada. Inserindo...
1 DOUTRINA MILITAR TERRESTRE entidade

Todas entidades já foram processadas...
{0: 1}

Relações:


ℹ Could not determine any instances in doc - returning doc as is.

Relações:



Entidades:
	Verificando: PODER DE COMBATE entidade
PODER DE COMBATE não encontrada. Inserindo...
2 PODER DE COMBATE entidade
	Verificando: poder de combate entidade
poder de combate não encontrada. Inserindo...
3 poder de combate e

In [ ]:
import re
# Função para retirar caracteres indesejáveis
def limpar_texto(text, undescore):
  text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
  text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
  text = re.sub(r"[^a-zA-Z.!?']", ' ', text)
  text = re.sub(r" +", ' ', text)
  text = text.lower()
  if undescore==1:
    text = re.sub(r" ",'_',text)
  return text

  x="correferencia_de"
  print(limpar_texto(x,1))

In [ ]:
#***************************************************************************************************************************
#Passo 6: Gerar os arquivos de saída (entidades.n3 e relacoes.n3): com as entidades e relações
#***************************************************************************************************************************


sujeito1 = "<http://bdex.eb.mil.br/jspui/classes/"
predicado1 = "<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> "
objeto1 = "<http://bdex.eb.mil.br/#schema> ."

# Processar as relações encontradas e gerar o arquivo entidades.n3
ent_n3=[]

if len(dim_ents) > 0:
  for ent in dim_ents:
    aux = sujeito1 + limpar_texto(ent[1],1) + "> " + predicado1 + objeto1
    ent_n3.append(aux)
#print(ent_n3)

rels_n3=[]
sujeito2 = "<http://bdex.eb.mil.br/jspui/propriedades/"
objeto2 = " <http://www.w3.org/1999/02/22-rdf-syntax-ns#Property> ."

# Processar as relações encontradas e gerar o arquivo relacoes.n3
if len(dim_rels)>0:
    for rels in dim_rels:
      aux2 = sujeito2 + limpar_texto(rels[2],1)+ "> " + predicado1 + objeto2
      aux3 = sujeito1 + limpar_texto(rels[1],1) + "> " + sujeito2 + limpar_texto(rels[2],1)+ "> " + sujeito1 + limpar_texto(rels[3],1) + "> ."
      rels_n3.append(aux2)
      rels_n3.append(aux3)
print(dim_rels)

saida = pd.DataFrame(ent_n3)
saida.to_csv('entidades.n3', header=None, sep=';', index=False)
saida.head()

saida2 = pd.DataFrame(rels_n3)
saida2.to_csv('relacoes.n3', header=None, sep=';', index=False)
saida2.head()



[[0, 'poder de combate', 'correferencia_de', 'PODER DE COMBATE'], [1, 'F Ter', 'aplicado_em', 'guerra'], [2, 'F Ter', 'aplicado_em', 'não guerra'], [3, 'Funções de Combate', 'tipo_de', 'PODER DE COMBATE'], [4, 'Comando e Controle', 'tipo_de', 'Funções de Combate'], [5, 'Movimento e Manobra', 'tipo_de', 'Funções de Combate'], [6, 'Inteligência', 'tipo_de', 'Funções de Combate'], [7, 'Fogos', 'tipo_de', 'Funções de Combate'], [8, 'Logística', 'tipo_de', 'Funções de Combate'], [9, 'Proteção', 'tipo_de', 'Funções de Combate']]


,0
0,<http://bdex.eb.mil.br/jspui/propriedades/corr...
1,<http://bdex.eb.mil.br/jspui/classes/poder_de_...
2,<http://bdex.eb.mil.br/jspui/propriedades/apli...
3,<http://bdex.eb.mil.br/jspui/classes/f_ter> <h...
4,<http://bdex.eb.mil.br/jspui/propriedades/apli...
